# **CHAPTER 12**
# **Custom Models and Training with TensorFlow**

**A Quick Tour of TensorFlow**
This subchapter introduces TensorFlow as a comprehensive framework for building, training, and deploying machine learning models. TensorFlow provides low-level operations as well as high-level APIs such as Keras. The chapter emphasizes that TensorFlow is not only a neural network library but also a general numerical computation platform optimized for performance and scalability.
TensorFlow operations work on tensors, which are multidimensional arrays similar to NumPy arrays but with additional features such as GPU acceleration and automatic differentiation.


**Using TensorFlow like NumPy**

This section explains that TensorFlow tensors behave similarly to NumPy arrays. They support indexing, slicing, reshaping, and broadcasting. However, tensors are immutable, meaning their values cannot be changed directly.
Conversion between NumPy arrays and TensorFlow tensors is straightforward, allowing easy integration with existing NumPy-based workflows.


In [2]:
import tensorflow as tf

# Membuat matrix 2x3
matrix = tf.constant([[1., 2., 3.],
                      [4., 5., 6.]])
print(matrix)


tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [12]:
import numpy as np
import tensorflow as tf

# Buat array NumPy
a = np.array([2., 4., 5.])

# Ubah menjadi tensor TensorFlow
tensor_a = tf.constant(a)
print(tensor_a)


tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)


In [13]:
t.numpy() # or np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [14]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [15]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [17]:
import tensorflow as tf

tf.constant(2., dtype=tf.float32) + tf.constant(40., dtype=tf.float32)


<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [19]:
import tensorflow as tf

# Pilih float32
result = tf.constant(2., dtype=tf.float32) + tf.constant(40., dtype=tf.float32)
print(result)  # 42.0, dtype=float32

# Atau pilih float64
result = tf.constant(2., dtype=tf.float64) + tf.constant(40., dtype=tf.float64)
print(result)  # 42.0, dtype=float64


tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(42.0, shape=(), dtype=float64)


In [20]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [22]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [23]:
v.assign(2 * v) # => [[2., 4., 6.], [8., 10., 12.]]
v[0, 1].assign(42) # => [[2., 42., 6.], [8., 10., 12.]]
v[:, 2].assign([0., 1.]) # => [[2., 42., 0.], [8., 10., 1.]]
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
# => [[100., 42., 0.], [8., 10., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

**Custom Loss Functions**

This subchapter explains situations where built-in loss functions are insufficient. TensorFlow allows users to define custom loss functions using Python functions that operate on tensors.
Custom loss functions must:
•	Take y_true and y_pred as inputs
•	Return a scalar tensor


In [24]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [33]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Load dataset
iris = load_iris()
X = iris.data      # fitur, shape (150, 4)
y = iris.target    # label, shape (150,)

# 2. Split dataset menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Standarisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # fit pada data training
X_test_scaled  = scaler.transform(X_test)       # transform data test


**Saving and Loading Models That Contain Custom Components**

Models that use custom loss functions or layers require special handling when saving and loading. TensorFlow allows custom objects to be passed explicitly during model loading.


In [38]:
import tensorflow as tf
from tensorflow import keras

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model = keras.models.Sequential([
    keras.layers.Dense(32, activation="relu", input_shape=(4,)),  # ganti 4 sesuai jumlah fitur
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1)  # regresi
])

model.compile(loss=create_huber(2.0), optimizer="nadam")

import numpy as np
X_train = np.random.rand(100, 4)
y_train = np.random.rand(100, 1)

model.fit(X_train, y_train, epochs=5)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.2460
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2005
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1613
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1314
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1073 


In [39]:
import tensorflow as tf
from tensorflow import keras

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


**Custom Activation Functions, Initializers, Regularizers, and Constraints**

TensorFlow allows customization of many internal components of neural networks. Users can define custom activation functions, weight initializers, regularizers, and constraints as simple functions.


In [40]:
import tensorflow as tf

def my_softplus(z):
    """Custom softplus activation function."""
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    """Custom Glorot/Xavier initializer."""
    stddev = tf.sqrt(2.0 / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    """Custom L1 regularization with coefficient 0.01."""
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    """Ensure weights are non-negative."""
    return tf.where(weights < 0.0, tf.zeros_like(weights), weights)


In [41]:
layer = keras.layers.Dense(30, activation=my_softplus,
kernel_initializer=my_glorot_initializer,
kernel_regularizer=my_l1_regularizer,
kernel_constraint=my_positive_weights)

In [43]:
import tensorflow as tf
from tensorflow import keras

class MyL1Regularizer(keras.regularizers.Regularizer):
    """Custom L1 regularizer with adjustable factor."""

    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        """Compute L1 regularization penalty."""
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        """Return config for serialization."""
        return {"factor": self.factor}


**Custom Metrics**

Metrics differ from loss functions in that they are used only for evaluation, not optimization. TensorFlow allows users to define custom metrics using stateful objects.


In [44]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [45]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.800000011920929>

In [46]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [47]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [48]:
precision.variables

[<Variable path=precision/true_positives, shape=(1,), dtype=float32, value=[4.]>,
 <Variable path=precision/false_positives, shape=(1,), dtype=float32, value=[4.]>]

In [52]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)  # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}


**Custom Layers**

This section explains how to create custom layers by subclassing keras.layers.Layer. Custom layers allow full control over forward computation and trainable parameters.


In [53]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [54]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel",
            shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal"
        )
        self.bias = self.add_weight(
            name="bias",
            shape=[self.units],
            initializer="zeros"
        )
        super().build(batch_input_shape)  # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "units": self.units,
            "activation": keras.activations.serialize(self.activation)
        }


In [55]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]

    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        # Mengembalikan daftar TensorShape untuk tiap output
        return [b1, b1, b1]  # catatan: ini sederhana, sebaiknya menyesuaikan broadcasting rules


In [56]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape


**Custom Models**

Custom models provide full flexibility by subclassing keras.models.Model. This approach is useful for complex architectures that cannot be expressed using Sequential or Functional APIs.


In [57]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                n_neurons,
                activation="elu",
                kernel_initializer="he_normal"
            ) for _ in range(n_layers)
        ]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z


In [58]:
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(
            30,
            activation="elu",
            kernel_initializer="he_normal"
        )
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        # menerapkan block residual beberapa kali
        for _ in range(1 + 3):
            Z = self.block1(Z)
            Z = self.block2(Z)
        return self.out(Z)


**Losses and Metrics Based on Model Internals**

When a model has multiple Internals, each output can have its own loss function and metrics. This allows different objectives to be optimized simultaneously.


In [59]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                30,
                activation="selu",
                kernel_initializer="lecun_normal"
            )
            for _ in range(5)
        ]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)


**Custom Training Loops**

This subchapter introduces custom training loops using tf.GradientTape. Custom loops provide maximum flexibility for research and experimentation.


In [60]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
kernel_regularizer=l2_reg),
keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [61]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [63]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics_str = " - ".join(
        ["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])]
    )
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics_str, end=end)


In [65]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size

optimizer = keras.optimizers.Nadam(learning_rate=0.01)  # gunakan learning_rate
loss_fn = keras.losses.MeanSquaredError()               # gunakan instance loss
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]


In [67]:
def random_batch(X, y, batch_size=32):
    idx = np.random.choice(len(X), size=batch_size, replace=True)
    return X[idx], y[idx]


**Summary**

Chapter 12 emphasizes flexibility and control in TensorFlow:
•	Custom losses, metrics, layers, and models
•	Advanced training workflows
•	Low-level and high-level API integration
This chapter prepares readers to build highly customized and production-ready deep learning systems.
